In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from collections import defaultdict
from joblib import Parallel, delayed

import sqlite3
import sys
import time
import math
#import tqdm
from tqdm.auto import tqdm
import datetime
import os
import pickle
from pathlib import Path

from glicko2 import Player
import multiprocessing

tqdm.pandas()

if os.path.exists('/workspace/data'):
    # Load the dictionary of DataFrames from the pickle
    data_path = '/workspace/data/'
else:
    data_path = '../data/'

## Load the single-set model saved from earlier

In [ ]:
from sklearn.metrics import accuracy_score, log_loss

dataset_df = pd.read_pickle(data_path + 'dataset_full.pkl')
dataset_df.sort_index(inplace=True) # For convenience, mostly. Not really necessary, we use .loc[] anyways

# Note that there is no real reason to keep separately computing the individual probabilities of winning each individual set.
# Let's just compute them all at once here.

single_set_model = None
with open(data_path + 'single_set_model.pkl', 'rb') as f:
    single_set_model = pickle.load(f)

# Make sure these match up with what features the model was trained on.
features_all_everything = ['p1_default_elo', 'p2_default_elo', 'p1_default_rd', 'p2_default_rd',
       'p1_default_updates', 'p2_default_updates', 'matchup_1', 'matchup_2',
       'matchup_3', 'matchup_4', 'matchup_5', 'matchup_6', 'matchup_7',
       'matchup_8', 'matchup_9', 'matchup_10', 'p1_m1_usage', 'p2_m1_usage',
       'p1/m1/m1_alt2_elo', 'p1/m1/m1_alt2_rd', 'p1/m1/m1_alt2_updates',
       'p2/m1/m1_alt2_elo', 'p2/m1/m1_alt2_rd', 'p2/m1/m1_alt2_updates',
       'p1/m1_alt3_elo', 'p1/m1_alt3_rd', 'p1/m1_alt3_updates',
       'p2/m1_alt3_elo', 'p2/m1_alt3_rd', 'p2/m1_alt3_updates']

dataset_df['p1_win_prob'] = single_set_model.predict_proba(dataset_df[features_all_everything])[:,1]

# As a sanity check, let's verify the accuracy and log loss on 2024 data
# Total accuracy and log loss (including lots of data that the model was trained on)
date_filter = (dataset_df['start'] >= datetime.datetime(2024,1,1)) & (dataset_df['end'] <= datetime.datetime(2024,12,31))
print("2024 single-set performance metrics")
print()
print("On all sets:")
print("Log loss: ", round(log_loss(dataset_df[date_filter]['winner'], dataset_df[date_filter]['p1_win_prob']), 3))
print("Accuracy: ", round(100.0 * accuracy_score(dataset_df[date_filter]['winner'], dataset_df[date_filter]['p1_win_prob'] >= 0.5), 1))
print()
print("Restricting to top 8 sets only:")
print("Log loss: ", round(log_loss(dataset_df[date_filter & dataset_df['top_8']]['winner'], dataset_df[date_filter & dataset_df['top_8']]['p1_win_prob']), 3))
print("Accuracy: ", round(100.0 * accuracy_score(dataset_df[date_filter & dataset_df['top_8']]['winner'], dataset_df[date_filter & dataset_df['top_8']]['p1_win_prob'] >= 0.5), 1))

## Load and clean up the tournament dataframe

In particular, this dataframe contains info on all top 8 players, along with the paths that they took to get to where they are in the tournament.

In [ ]:
tournament_df = pd.read_pickle(data_path + 'top_8_tournament_previous_sets_and_results_with_winners_df')

# Filter down to tournaments which actually have valid top 8 data, and previous data on getting there.
tournament_df = tournament_df.loc[tournament_df[['LN_A_p1', 'LN_A_p2', 'LN_B_p1', 'LN_B_p2',
                                                 'WSF_A_p1', 'WSF_A_p2', 'WSF_B_p1', 'WSF_B_p2',
                                                 'LN_A_p1_non_top_8_sets', 'LN_A_p2_non_top_8_sets',
                                                 'LN_B_p1_non_top_8_sets', 'LN_B_p2_non_top_8_sets',
                                                 'WSF_A_p1_non_top_8_sets', 'WSF_A_p2_non_top_8_sets',
                                                 'WSF_B_p1_non_top_8_sets', 'WSF_B_p2_non_top_8_sets']].dropna().index]


# Very rarely (not sure where the problem is) you get something not actually in the single-set dataframe
# It is not actually that common though, so let's just delete those instances.
def references_valid_sets(prev_sets):
    for x in prev_sets:
        if x[0] not in dataset_df.index:
            return False
        
    return True

filter = tournament_df[['LN_A_p1_non_top_8_sets', 'LN_A_p2_non_top_8_sets',
                        'LN_B_p1_non_top_8_sets', 'LN_B_p2_non_top_8_sets',
                        'WSF_A_p1_non_top_8_sets', 'WSF_A_p2_non_top_8_sets',
                        'WSF_B_p1_non_top_8_sets', 'WSF_B_p2_non_top_8_sets']].map(references_valid_sets).all(axis=1)

tournament_df = tournament_df[filter]

# Likewise, some of these sets don't seem to have a valid winner
tournament_df = tournament_df[~tournament_df['winner_id'].isna()]

# A bit more cleanup, for sanity
min_date = datetime.datetime(2015,1,1)
max_date = datetime.datetime(2024,12,31)

tournament_df = tournament_df[(tournament_df['start'] >= min_date) &
                              (tournament_df['end'] >= min_date) &
                              (tournament_df['start'] <= max_date) &
                              (tournament_df['end'] <= max_date)]

# We will only be dealing with data from 2023 onwards, because the single-set predictor that we will be using
# was trained on data up to the end of 2022, and we don't want it leaking data.
# This will also speed up computations by not performing them on data we don't care about.
tournament_df = tournament_df[tournament_df['start'] >= datetime.datetime(2023,1,1)]

tournament_df.sort_values(by=['end', 'start'], inplace=True)

tournament_df[['winner_id', 'LN_A_p1', 'LN_A_p2', 'LN_B_p1', 'LN_B_p2', 'WSF_A_p1', 'WSF_A_p2', 'WSF_B_p1', 'WSF_B_p2']]

## A bit more feature engineering

Here, we compute a "score" that shows how well a player is doing relative to their skill level and the skill level of their opponents in this tournament. If they are beating players well beyond their normal skill level, their score goes up. If they lose, it goes down.

More specifically, we compute the probability that they win, and add the negative log of that probability if they indeed win. Losses are similar, but based on the probability that they lose, and subtracted instead.

In [ ]:
# Compute a score based on how likely it was that they actually made it to the top.
# A sort of sum of negative log probabilities, where wins are positive and losses are negative
def prev_set_score(prev_sets):
    result = 0

    for x in prev_sets:
        data = dataset_df.loc[x[0], ['winner', 'p1_win_prob']]
        outcome = x[1]

        # We don't know if this player is p1 or p2 in this list, but this can determine it without looking at player id
        # Compare if (player we are interested in wins) vs (did p1 win)
        if outcome == (data['winner'] == 1.0): # The player is p1
            if outcome: # player wins, as p1
                result += (-np.log(data['p1_win_prob']))
            else:       # player loses, as p1
                result -= (-np.log(1-data['p1_win_prob']))
        else:                                  # The player is p2
            if outcome: # player wins, as p2
                result += (-np.log(1-data['p1_win_prob']))
            else:       # player loses, as p2
                result -= (-np.log(data['p1_win_prob']))

    return result

# Example data (note that there is a consistent ELO throughout this entire dataset)
prev_sets = tournament_df.iloc[10000]['LN_A_p1_non_top_8_sets']
print(prev_sets)
print()
print(dataset_df.loc[[x[0] for x in prev_sets], ['p1_default_elo', 'p2_default_elo', 'winner', 'p1_win_prob']])
print()
print("Previous set score: ", prev_set_score(tournament_df.iloc[10000]['LN_A_p1_non_top_8_sets']))

Generally, we will keep track of all of the necessary data for each player in the top 8 by keeping track of what their starting position was.

In [ ]:
top_8_pos = ['LN_A_p1', 'LN_A_p2', 'LN_B_p1', 'LN_B_p2', 'WSF_A_p1', 'WSF_A_p2', 'WSF_B_p1', 'WSF_B_p2']
top_8_prevs = [x + '_non_top_8_sets' for x in top_8_pos]
top_8_prevs_lengths = [x + "_len" for x in top_8_prevs] # These columns will just keep track of how many sets the player went through to get to the top 8
top_8_prevs_scores = [x + "_score" for x in top_8_prevs] # These will keep track of their "score" that shows how "well" they are performing relative to their predicted odds.

tournament_df[top_8_prevs_lengths] = tournament_df[top_8_prevs].map(lambda x: len(x)).to_numpy()
tournament_df[top_8_prevs_scores] = tournament_df[top_8_prevs].map(prev_set_score).to_numpy()
tournament_df[top_8_prevs + top_8_prevs_lengths + top_8_prevs_scores].head()

In [ ]:
# Create a column that contains the top 8 starting position of the winner
# (or more specifically, the numeric index in top_8_pos)

tournament_df['winner_index'] = 8 # Dummy value, has to be 0-7

for i,position in enumerate(top_8_pos):
    found_filter = (tournament_df['winner_id'] == tournament_df[top_8_pos[i]])
    tournament_df.loc[found_filter, 'winner_index'] = i

(tournament_df['winner_index'] == 8).sum() # Should be zero (everything found)

Start pulling in data (like ELO) for each of the top 8 players in the tournaments.

In [ ]:
# We will need to have all of the features we've engineered for each of the players that made it to the top 8
# We can be clever and pull most of them (elo-based features) from the previous sets in the dataframe
# The rest (player vs player stats, also called "matchup", but might be renamed) have to be pulled in manually

features_elo = ['p1_default_elo', 'p2_default_elo', 'p1_default_rd', 'p2_default_rd',
       'p1_default_updates', 'p2_default_updates', 'p1_m1_usage', 'p2_m1_usage',
       'p1/m1/m1_alt2_elo', 'p1/m1/m1_alt2_rd', 'p1/m1/m1_alt2_updates',
       'p2/m1/m1_alt2_elo', 'p2/m1/m1_alt2_rd', 'p2/m1/m1_alt2_updates',
       'p1/m1_alt3_elo', 'p1/m1_alt3_rd', 'p1/m1_alt3_updates',
       'p2/m1_alt3_elo', 'p2/m1_alt3_rd', 'p2/m1_alt3_updates']

features_matchup = ['matchup_1', 'matchup_2', 'matchup_3', 'matchup_4', 'matchup_5',
                    'matchup_6', 'matchup_7', 'matchup_8', 'matchup_9', 'matchup_10']

In [ ]:
# In order to avoid an organizational nightmare, each of the following:
#     'LN_A_p1', 'LN_A_p2', 'LN_B_p1', 'LN_B_p2', 'WSF_A_p1', 'WSF_A_p2', 'WSF_B_p1', 'WSF_B_p2'
# will have their data stored in separate dataframes, each held in the following dictionary

top_8_stats = {}

def pull_data_from_set(loc, outcome):
    set_data = dataset_df.loc[loc]
    player_num = 'p1' if outcome == (set_data['winner'] == 1.0) else 'p2' # Sneaky way of getting the player number

    features_to_pull = [x for x in features_elo if player_num in x]
    pulled_data = set_data[features_to_pull].copy()
    pulled_data.index = [x.replace(player_num, '') for x in pulled_data.index] # We will add player numbers on an as-needed basis later

    return pulled_data

for top_8_position in tqdm(top_8_pos):
    # First, pull in player 1 data from a previous match.
    # Note that the player might NOT be player 1 in the match that we are pulling from
    top_8_stats[top_8_position] = tournament_df[top_8_position + '_non_top_8_sets'].apply(lambda x: pull_data_from_set(x[0][0], x[0][1]))

The most important information about the players in the top 8 is essentially how likely each player is to win against the other player. We can use our single-set predictor to estimate this. We can also make use of the information on how well each player has been doing in the tournament relative to their skill level/opponent skill levels, possibly in order to "adjust" the aforementioned probabilities. Or just to toss it as extra information in whatever ML model.

In [ ]:
# First, let's compute pairwise probabilities of one player in the top 8 winning against another player
def compute_pairwise_prob(row):
    # Row represents p1, column represents p2 (or specifically, the index in top_8_pos). Always follows this order:
    # 'LN_A_p1', 'LN_A_p2', 'LN_B_p1', 'LN_B_p2', 'WSF_A_p1', 'WSF_A_p2', 'WSF_B_p1', 'WSF_B_p2'
    pairwise_probs = np.zeros(shape=(8,8))
    
    # For convenience, put everything into one dataframe and then run the single set model
    # This is probably more efficient than doing things line by line
    combination_stats = []

    for r in range(0,8):
        for c in range(0,8):
            p1_data = top_8_stats[top_8_pos[r]].loc[row.name]
            p1_data.index = ['p1' + x for x in p1_data.index]

            p2_data = top_8_stats[top_8_pos[c]].loc[row.name]
            p2_data.index = ['p2' + x for x in p2_data.index]

            #TODO: Actually populate this with proper data!
            #      This is currently only just placeholder data,
            #      indicating that the players have never played together before (0.5).
            matchup_data = pd.Series(0.5, index=['matchup_' + str(n) for n in range(1,10+1)])

            total_data = pd.concat([p1_data, p2_data, matchup_data])
            total_data = total_data[features_all_everything] # Entries need to be in the correct order

            combination_stats.append(total_data)

    combination_stats = pd.DataFrame(combination_stats)

    y_prob = single_set_model.predict_proba(combination_stats)

    # Now actually populate this probability matrix with data.
    # Note that we can just use the same nested loop and read off the entries of the 1D probability array one at a time.
    # This will put things in the correct order.
    i = 0
    for r in range(0,8):
        for c in range(0,8):
            pairwise_probs[r,c] = y_prob[i,1]
            i += 1

    # NOTE: Some models that we train are highly non-symmetric, even though they very much should be,
    #       given that we have randomized the players. We can fix that issue here.
    pairwise_probs = 0.5 * (pairwise_probs + (1 - pairwise_probs.T))

    return pairwise_probs.flatten()

pairwise_prob = tournament_df.apply(compute_pairwise_prob, axis=1)
pairwise_prob = np.stack(pairwise_prob.to_numpy()) # Fixes "single column of np arrays" nonsense

# Add that data as columns in the original dataframe
pairwise_prob_cols = []
for r in range(0,8):
    for c in range(0,8):
        pairwise_prob_cols.append("pairprob/" + top_8_pos[r] + "/" + top_8_pos[c])

tournament_df[pairwise_prob_cols] = pairwise_prob

tournament_df[pairwise_prob_cols]

In [ ]:
# Add that data as columns in the original dataframe

# Note that we only really need the strictly lower triangular part of the probability matrix
# Everything else is redundant (the diagonal is 0.5, upper triangular part is 1 - lower triangular)
# The entire matrix is kept in there just for easy reading.

pairwise_prob_cols = []
pairwise_prob_cols_reduced = []

for r in range(0,8):
    for c in range(0,8):
        col_name = "pairprob/" + top_8_pos[r] + "/" + top_8_pos[c]

        pairwise_prob_cols.append(col_name)

        if r > c:
            pairwise_prob_cols_reduced.append(col_name)

tournament_df[pairwise_prob_cols] = pairwise_prob
tournament_df[pairwise_prob_cols_reduced]

Finally, we can (disclaimer: slightly inaccurately) compute all possible paths throughout the top 8, and the probability of winning along each path.

In [ ]:
# A slighltly incorrectly implemented algorithm that goes through all possible paths throughout the top 8,
# and computes the corresponding probabilities of each player making it through.
def compute_path_prob(row):
    pairwise_probs = row[pairwise_prob_cols].to_numpy().astype(float).reshape((8,8))
    pairwise_probs_zero_diagonal = pairwise_probs - 0.5 * np.identity(8) # Used for janky computations

    # Now start building the tree structure of how the tournament can play out.
    # Each "cell" will represent some set in the tournament played by some p1 and p2.
    # The cell will have to keep track of all of the probabilities of each player making it to that point.
    #
    # Links should have the form (cell, 'winner') or (cell, 'loser'),
    # describing if it is the winner or the loser of the previous that gets to this one
    class cell:
        def __init__(self, p1=None, p2=None, p1_link=None, p2_link=None):
            if p1==None:
                self.p1_probs = None
            else:
                self.p1_probs = np.zeros(8)
                self.p1_probs[p1] = 1.0

            if p2==None:
                self.p2_probs = None
            else:
                self.p2_probs = np.zeros(8)
                self.p2_probs[p2] = 1.0

            # Links to previous cells
            self.p1_link = p1_link
            self.p2_link = p2_link

            # Used for a (hopefully) temporary patch on the fact that these computations are not entirely accurate
            self.pairwise_probs_zero_diagonal = pairwise_probs - 0.5 * np.identity(8)

        # Get the probabilities from the previous cell.
        # Should not be called if there are no links to previous cells.
        def fetch_probs(self):
            self.p1_probs = self.p1_link[0].compute_winner_probs() if self.p1_link[1] == 'winner' else self.p1_link[0].compute_loser_probs()
            self.p2_probs = self.p2_link[0].compute_winner_probs() if self.p2_link[1] == 'winner' else self.p2_link[0].compute_loser_probs()
        
        # Probability of making it to this cell, and then proceeding to win
        def compute_winner_probs(self):
            if self.p1_probs is None or self.p2_probs is None:
                self.fetch_probs()

            probs = np.zeros(8)

            # Old code, far less efficient. Might make the numpy operations make sense though.
            '''
            for p1 in range(0,8):
                # Save a result for p1.
                # It will be the sum over all p2 of
                # (probability that p1 got there) * (probability that p2 got there) * (probability p1 beats p2)
                for p2 in range(0,8):
                    probs[p1] += self.p1_probs[p1] * self.p2_probs[p2] * pairwise_probs[p1, p2]
                    probs[p2] += self.p1_probs[p1] * self.p2_probs[p2] * (1.0 - pairwise_probs[p1, p2])
            '''
            # Just remember that 1-pairwise_probs is the transpose of pairwise_probs, by symmetry
            #
            # TODO: I just realized that the probability of a certain player becoming p1 and another becoming p2 are NOT independent.
            #       In particular, these probabilities become correlated when you could potentially have the same player as p1 or p2.
            #       This is a bit of a janky patch that hopefully gives accurate enough probabilities, but we should come up with a proper fix.       
            probs += self.p1_probs * (pairwise_probs_zero_diagonal @ self.p2_probs) # Probability that (specific p1) wins
            probs += self.p2_probs * (pairwise_probs_zero_diagonal @ self.p1_probs) # Same but p2

            probs /= probs.sum() # Purely due to zeroing out the diagonal of pairwise_probs

            return probs

        # Probability of making it to this cell, and then proceeding to lose
        def compute_loser_probs(self):
            if self.p1_probs is None or self.p2_probs is None:
                self.fetch_probs()

            probs = np.zeros(8)

            '''
            for p1 in range(0,8):
                # Same, except use probability of p1 losing
                for p2 in range(0,8):
                    probs[p1] += self.p1_probs[p1] * self.p2_probs[p2] * (1.0 - pairwise_probs[p1, p2])
                    probs[p2] += self.p1_probs[p1] * self.p2_probs[p2] * pairwise_probs[p1, p2]
            '''
            # TODO: Same janky patch as in winners case here.
            probs += self.p1_probs * (pairwise_probs_zero_diagonal.T @ self.p2_probs)
            probs += self.p2_probs * (pairwise_probs_zero_diagonal.T @ self.p1_probs)

            probs /= probs.sum()
             
            return probs
        
    # 'LN_A_p1', 'LN_A_p2', 'LN_B_p1', 'LN_B_p2', 'WSF_A_p1', 'WSF_A_p2', 'WSF_B_p1', 'WSF_B_p2'
    WSFA = cell(p1=4, p2=5)
    WSFB = cell(p1=6, p2=7)
    LNA  = cell(p1=0, p2=1)
    LNB  = cell(p1=2, p2=3)

    WF = cell(p1_link=(WSFA, 'winner'), p2_link=(WSFB, 'winner'))

    LQFA = cell(p1_link=(WSFA, 'loser'), p2_link=(LNA, 'winner'))
    LQFB = cell(p1_link=(WSFB, 'loser'), p2_link=(LNB, 'winner'))

    LSF = cell(p1_link=(LQFA, 'winner'), p2_link=(LQFB, 'winner'))

    LF = cell(p1_link=(WF, 'loser'), p2_link=(LSF, 'winner'))

    GF = cell(p1_link=(WF, 'winner'), p2_link=(LF, 'winner'))

    # From the Grand Final onwards, some special cases are required, due to how the Grand Final Reset works
    GF.fetch_probs()

    # TODO: Again, same janky fix as before, "removing" correlation between p1 and p2
    win_as_p1_probs = GF.p1_probs * (pairwise_probs_zero_diagonal @ GF.p2_probs) # direct win as p1 (WF winner)
    win_as_p1_probs += GF.p1_probs * ((pairwise_probs_zero_diagonal.T * pairwise_probs_zero_diagonal) @ GF.p2_probs) # p2 win, then p1 win in GFR

    win_as_p2_probs = GF.p2_probs * ((pairwise_probs_zero_diagonal ** 2) @ GF.p1_probs) # win by 2 required for LF winner

    probs = win_as_p1_probs + win_as_p2_probs
    probs /= probs.sum() # Again due to that janky fix

    return probs

## Benchmarking and baselines

Here, we start comparing the performance of a few "obvious" models, such as just choosing the person with the highest ELO out of the top 8 (or winners' side of the top 8), or simulating all possible paths throughout the top 8 and computing the probability of winning as a result.

In [ ]:
features = []
features += pairwise_prob_cols_reduced
features += top_8_prevs_lengths + top_8_prevs_scores

In [ ]:
# Baseline of "who has the higher elo"

def pull_elo_from_set(loc, outcome):
    set_data = dataset_df.loc[loc]
    player_num = 'p1' if outcome == (set_data['winner'] == 1.0) else 'p2' # Sneaky way of getting the player number

    feature_to_pull = player_num + '_default_elo'
    pulled_data = set_data[feature_to_pull]

    return pulled_data

# First, pull in player 1 data from a previous match.
# Note that the player might NOT be player 1 in the match that we are pulling from
tournament_df[[x + '_elo' for x in top_8_pos]] = tournament_df[[x + '_non_top_8_sets' for x in top_8_pos]].map(lambda x: pull_elo_from_set(x[0][0], x[0][1])).to_numpy()

tournament_df['elo_prediction'] = tournament_df[[x + '_elo' for x in top_8_pos]].idxmax(axis=1).apply(lambda x: x.replace('_elo', ''))
tournament_df['elo_prediction'] = tournament_df['elo_prediction'].apply(lambda x: top_8_pos.index(x))

tournament_df['elo_WSF_prediction'] = tournament_df[[x + '_elo' for x in top_8_pos if "WSF" in x]].idxmax(axis=1).apply(lambda x: x.replace('_elo', ''))
tournament_df['elo_WSF_prediction'] = tournament_df['elo_WSF_prediction'].apply(lambda x: top_8_pos.index(x))

In [ ]:
# Predictor of "who has the highest path probability, taking into account all possible paths on how the top 8 will play out"

result = tournament_df.apply(compute_path_prob, axis=1)
tournament_df = pd.concat([tournament_df, pd.DataFrame(np.stack(result.to_numpy()), index=result.index, columns=[x + '_winprob' for x in top_8_pos])], axis=1)

tournament_df['path_prediction'] = tournament_df[[x + '_winprob' for x in top_8_pos]].idxmax(axis=1).apply(lambda x: x.replace('_winprob', ''))
tournament_df['path_prediction'] = tournament_df['path_prediction'].apply(lambda x: top_8_pos.index(x))

In [ ]:
train_df = tournament_df[(tournament_df['start'] >= datetime.datetime(2023,1,1)) & (tournament_df['end'] <= datetime.datetime(2023,12,31))].copy()
test_df  = tournament_df[(tournament_df['start'] >= datetime.datetime(2024,1,1)) & (tournament_df['end'] <= datetime.datetime(2024,12,31))].copy()

In [ ]:
print("Train set baselines:")
print("Highest ELO out of top 8, accuracy:               ", round(100.0 * (train_df['winner_index'] == train_df['elo_prediction']).astype(float).mean(), 1))
print("Highest ELO out of WSF, accuracy:                 ", round(100.0 * (train_df['winner_index'] == train_df['elo_WSF_prediction']).astype(float).mean(), 1))
print("Computing all ways top 8 can play out, accuracy:  ", round(100.0 * (train_df['winner_index'] == train_df['path_prediction']).astype(float).mean(), 1))

## A more advanced model and hyperparameter tuning

Just because choosing the highest ELO seems to be such a powerful predictor to begin with, we will make sure to add those to the list of features that we will use. We won't add any of the other old engineered features, because the model already has a bunch of new "fancier" engineered features, and too many might make it perform worse.

**NOTE:** Hyperparameter tuning trials was reduced down to a very low number, so that this entire notebook could be run in a reasonable amount of time. Optimal parameters from a proper number of trials have already been found and are already provided after the hyperparameter tuning block.

In [ ]:
features += [x + '_elo' for x in top_8_pos]

In [ ]:
# Some features should NOT be scaled, like probabilities.
# Let's just manually scale ELO (note that we will be using regularization)
if tournament_df[[x + '_elo' for x in top_8_pos][0]].mean() >= 400.0: # Prevents this from accidentally being run twice.
    print("Scaling ELO!")
    tournament_df[[x + '_elo' for x in top_8_pos]] /= 1500.0

In [ ]:
# Perform hyperparameter tuning on XGBoost
import xgboost as xgb
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import optuna

def objective(trial):
    max_depth        = trial.suggest_int("max_depth", 2, 10, step=1)
    learning_rate    = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
    n_estimators     = trial.suggest_int("n_estimators", 50, 1000, step=50)
    subsample        = trial.suggest_float("subsample", 0.6, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.6, 1.0)
    gamma            = trial.suggest_float("gamma", 0.0, 5.0)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 10)
    reg_lambda       = trial.suggest_float("lambda", 1e-3, 10.0, log=True)
    reg_alpha        = trial.suggest_float("alpha", 1e-3, 10.0, log=True)

    model = xgb.XGBClassifier(max_depth=max_depth,
                              learning_rate=learning_rate,
                              n_estimators=n_estimators,
                              subsample=subsample,
                              colsample_bytree=colsample_bytree,
                              gamma=gamma,
                              min_child_weight=min_child_weight,
                              reg_lambda=reg_lambda,
                              reg_alpha=reg_alpha)
    
    # The percentage of people that win the tournament when starting from losers side is very small, but not zero
    # Hence, it is probably good to use a stratified k-fold split for cross-validation

    n_splits = 3

    skf = StratifiedKFold(n_splits=n_splits, random_state=0, shuffle=True)
    results = np.zeros(n_splits)
    
    for i, (train_index, val_index) in enumerate(skf.split(train_df[features], train_df['winner_index'])):
        model.fit(train_df.iloc[train_index][features], train_df.iloc[train_index]['winner_index'])

        y_pred = model.predict(train_df.iloc[val_index][features])
        results[i] = accuracy_score(train_df.iloc[val_index]['winner_index'], y_pred)
    
    return results.mean()

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10, timeout=1800)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))


In [ ]:
# Plucked from the above hyperparameter tuning session.
# Doesn't really seem to be getting much better.
xgb_tuned = xgb.XGBClassifier(max_depth=5,
                              learning_rate=0.013278022811244645,
                              n_estimators=300,
                              subsample=0.7451791331241274,
                              colsample_bytree=0.7785615347277948,
                              gamma=2.819394518381603,
                              min_child_weight=6,
                              reg_lambda=0.41270859938111326,
                              reg_alpha=1.9859484482942584)

xgb_tuned.fit(train_df[features], train_df['winner_index'])
print("Accuracy of XGBoost on test set: ", round(100.0 * accuracy_score(test_df['winner_index'], xgb_tuned.predict(test_df[features])), 1))

In [ ]:
print("Test set baselines:")
print()
print("Highest ELO out of top 8, accuracy:               ", round(100.0 * (test_df['winner_index'] == test_df['elo_prediction']).astype(float).mean(), 1))
print("Highest ELO out of WSF, accuracy:                 ", round(100.0 * (test_df['winner_index'] == test_df['elo_WSF_prediction']).astype(float).mean(), 1))
print("Computing all ways top 8 can play out, accuracy:  ", round(100.0 * (test_df['winner_index'] == test_df['path_prediction']).astype(float).mean(), 1))